In [1]:
import MetaTrader5 as mt
from datetime import datetime
login=176839910
password='444444#'
server='Exness-MT5Trial7'

mt.initialize()

True

In [2]:
import pandas as pd

In [3]:
mt.login(login,password,server)

True

In [4]:
symbol="EURUSDm"
timeframe=mt.TIMEFRAME_M5
date_from=datetime(2024,10,27)
date_to=datetime.now()
date_to=datetime(2024,10,30)


df=pd.DataFrame(mt.copy_rates_range(symbol,timeframe,date_from,date_to))
df['DateTime'] = pd.to_datetime(df['time'], unit='s')
df=df.drop(columns=['tick_volume',	'spread','real_volume'])

In [5]:
df

,time,open,high,low,close,DateTime
0,1730063100,1.07919,1.07929,1.07879,1.07883,2024-10-27 21:05:00
1,1730063400,1.07883,1.07926,1.07883,1.07921,2024-10-27 21:10:00
2,1730063700,1.07921,1.07925,1.07849,1.07868,2024-10-27 21:15:00
3,1730064000,1.07865,1.07910,1.07864,1.07909,2024-10-27 21:20:00
4,1730064300,1.07910,1.07918,1.07856,1.07893,2024-10-27 21:25:00
...,...,...,...,...,...,...
141,1730105400,1.08141,1.08177,1.08138,1.08176,2024-10-28 08:50:00
142,1730105700,1.08179,1.08209,1.08174,1.08185,2024-10-28 08:55:00
143,1730106000,1.08187,1.08195,1.08171,1.08173,2024-10-28 09:00:00
144,1730106300,1.08173,1.08181,1.08152,1.08174,2024-10-28 09:05:00


In [6]:
df

,time,open,high,low,close,DateTime
0,1730063100,1.07919,1.07929,1.07879,1.07883,2024-10-27 21:05:00
1,1730063400,1.07883,1.07926,1.07883,1.07921,2024-10-27 21:10:00
2,1730063700,1.07921,1.07925,1.07849,1.07868,2024-10-27 21:15:00
3,1730064000,1.07865,1.07910,1.07864,1.07909,2024-10-27 21:20:00
4,1730064300,1.07910,1.07918,1.07856,1.07893,2024-10-27 21:25:00
...,...,...,...,...,...,...
141,1730105400,1.08141,1.08177,1.08138,1.08176,2024-10-28 08:50:00
142,1730105700,1.08179,1.08209,1.08174,1.08185,2024-10-28 08:55:00
143,1730106000,1.08187,1.08195,1.08171,1.08173,2024-10-28 09:00:00
144,1730106300,1.08173,1.08181,1.08152,1.08174,2024-10-28 09:05:00


In [7]:
#df=df.drop(columns=['datetime'])
from sklearn.base import BaseEstimator, TransformerMixin
import pandas_ta as ta
import pandas as pd
import numpy as np

In [8]:
df=df.drop(columns=['time'])

In [9]:


class rsi(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self


    def transform(self, X):
      
        X['body']=abs(X.close-X.open)
      
        X['ltail']=np.where(X.close > X.open,X.open-X.low,X.close-X.low)
        X['utail']= np.where(X.close > X.open,X.high-X.close,X.high-X.open)#np.where(df['close']>df['open'],df['high']-df['close'],df['high']-df['open'])
        
        
        long_utail = X['utail'] > 2 * abs(X['body'])  # Long upper tail (twice the body length)
        short_ltail = X['ltail'] < abs(X['body'])  # Short lower tail
     
        # Combine conditions to identify bearish pinbars
        X['bearish_pinbar'] = np.where(long_utail & short_ltail , 1, 0)
       

        

        # Combine conditions to identify bullish pinbars


        
        # Drop rows with NaN values (generated from the shifting)
        X = X.dropna()
        return X


In [10]:
#1:1
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
class generate_rsi_crossover_success1(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Reset the index to ensure clean iteration
        X = X.reset_index(drop=True)
        
        crossed_40 = False  # To track if RSI has crossed 40
        crossed_40_rsi = None  # RSI value at the point of crossing 40
        crossed_40_date = None  # DateTime when RSI crossed 40
        crossed_40_index = None  # To track the index where RSI crossed 40
        
        filtered_data = []  # List to hold the filtered data
        filtered_data1 = []
        filtered_data2 = []

        # Iterate over the rows of the dataframe
        for index, row in X.iterrows():
            # Check if RSI crosses above 40 and the previous value was below 40
            if index > 0 and row['close'] < X.loc[index - 1, 'low'] and not crossed_40 and X.loc[index - 1, 'bearish_pinbar'] == 1:
                
 
                crossed_40 = True
                crossed_40_index = index  # Track the index where RSI crossed 40
            
                lowu=(row['close']-0.0008)
                lowu1=(row['close']+0.0002)
                enp=row['close']
                DateTime=row['DateTime']

                
                
                filtered_data.append({
                    'entryprice':row['close'],
                    'DateTime':DateTime,
                        'entrytime': row['DateTime'], 
                'stoploss': lowu1,
                'target': lowu, 
            
                    })
               
            # Check if RSI drops below 30 and ensure it hasn't exceeded 60 since crossing 40
            elif crossed_40 and row['low'] < lowu :
                # Check that RSI didn't go above 60 since crossing 40
              
                crossed_40 = False
                filtered_data2.append({
                    'entryprice':enp,
                     'DateTime':DateTime,
                        'targettime': row['DateTime'], 
                'stoploss': lowu1,
                'target': lowu, 
            
                    })

               
            # If RSI goes above 60 after crossing 40, reset crossed_40
            elif crossed_40 and row['high'] > lowu1:
                crossed_40 = False
                filtered_data1.append({
                    'entryprice':enp,
                     'DateTime':DateTime,
                        'stoplosstime': row['DateTime'], 
                'stoploss': lowu1,
                'target': lowu, 
            
                    })
               
        # Convert filtered_data to a DataFrame and return it
        return pd.DataFrame(filtered_data),pd.DataFrame(filtered_data1),pd.DataFrame(filtered_data2)




In [11]:
def get_data_transformer_object()->Pipeline:
    try:

        robust_scaler = RobustScaler()
        simple_imputer = SimpleImputer(strategy="constant", fill_value=0)
        rsisma= rsi()
        generate_rsi_crosover_sucsess1= generate_rsi_crossover_success1()

        # data=self.get_rsi_and_sma()
        preprocessor = Pipeline(
            steps=[
               ("rsical", rsisma),
                ("cros", generate_rsi_crosover_sucsess1),
             
                #("namedrop",dropper),
                


                # keep every feature in same range and handle outlier
            ]
        )

        return preprocessor

    except Exception as e:
        raise e


In [13]:
df

,open,high,low,close,DateTime
0,1.07919,1.07929,1.07879,1.07883,2024-10-27 21:05:00
1,1.07883,1.07926,1.07883,1.07921,2024-10-27 21:10:00
2,1.07921,1.07925,1.07849,1.07868,2024-10-27 21:15:00
3,1.07865,1.07910,1.07864,1.07909,2024-10-27 21:20:00
4,1.07910,1.07918,1.07856,1.07893,2024-10-27 21:25:00
...,...,...,...,...,...
140,1.08122,1.08148,1.08120,1.08141,2024-10-28 08:45:00
141,1.08141,1.08177,1.08138,1.08176,2024-10-28 08:50:00
142,1.08179,1.08209,1.08174,1.08185,2024-10-28 08:55:00
143,1.08187,1.08195,1.08171,1.08173,2024-10-28 09:00:00


In [14]:
preprocessor = get_data_transformer_object()

preprocessor.fit(df)
#train_df1=train_df.tail(1000)
#train_df1


Pipeline(steps=[('rsical', rsi()), ('cros', generate_rsi_crossover_success1())])

In [15]:
a,b,c=preprocessor.transform(df)

In [16]:
b

,entryprice,DateTime,stoplosstime,stoploss,target
0,1.11323,2024-09-30 19:55:00,2024-09-30 20:20:00,1.11343,1.11243
1,1.11319,2024-09-30 21:15:00,2024-09-30 21:50:00,1.11339,1.11239
2,1.10861,2024-10-01 10:30:00,2024-10-01 10:35:00,1.10881,1.10781
3,1.10917,2024-10-01 11:10:00,2024-10-01 11:15:00,1.10937,1.10837
4,1.10736,2024-10-01 14:50:00,2024-10-01 14:55:00,1.10756,1.10656
...,...,...,...,...,...
64,1.07920,2024-10-24 11:45:00,2024-10-24 11:50:00,1.07940,1.07840
65,1.08222,2024-10-25 01:20:00,2024-10-25 01:45:00,1.08242,1.08142
66,1.08317,2024-10-25 14:05:00,2024-10-25 14:10:00,1.08337,1.08237
67,1.08018,2024-10-25 17:20:00,2024-10-25 17:25:00,1.08038,1.07938


In [17]:
a

,entryprice,DateTime,entrytime,stoploss,target
0,1.11323,2024-09-30 19:55:00,2024-09-30 19:55:00,1.11343,1.11243
1,1.11319,2024-09-30 21:15:00,2024-09-30 21:15:00,1.11339,1.11239
2,1.11085,2024-10-01 08:40:00,2024-10-01 08:40:00,1.11105,1.11005
3,1.10861,2024-10-01 10:30:00,2024-10-01 10:30:00,1.10881,1.10781
4,1.10917,2024-10-01 11:10:00,2024-10-01 11:10:00,1.10937,1.10837
...,...,...,...,...,...
89,1.08317,2024-10-25 14:05:00,2024-10-25 14:05:00,1.08337,1.08237
90,1.08018,2024-10-25 17:20:00,2024-10-25 17:20:00,1.08038,1.07938
91,1.07964,2024-10-25 19:20:00,2024-10-25 19:20:00,1.07984,1.07884
92,1.07964,2024-10-28 00:50:00,2024-10-28 00:50:00,1.07984,1.07884


In [18]:
a

,entryprice,DateTime,entrytime,stoploss,target
0,1.11323,2024-09-30 19:55:00,2024-09-30 19:55:00,1.11343,1.11243
1,1.11319,2024-09-30 21:15:00,2024-09-30 21:15:00,1.11339,1.11239
2,1.11085,2024-10-01 08:40:00,2024-10-01 08:40:00,1.11105,1.11005
3,1.10861,2024-10-01 10:30:00,2024-10-01 10:30:00,1.10881,1.10781
4,1.10917,2024-10-01 11:10:00,2024-10-01 11:10:00,1.10937,1.10837
...,...,...,...,...,...
89,1.08317,2024-10-25 14:05:00,2024-10-25 14:05:00,1.08337,1.08237
90,1.08018,2024-10-25 17:20:00,2024-10-25 17:20:00,1.08038,1.07938
91,1.07964,2024-10-25 19:20:00,2024-10-25 19:20:00,1.07984,1.07884
92,1.07964,2024-10-28 00:50:00,2024-10-28 00:50:00,1.07984,1.07884


In [19]:
c

,entryprice,DateTime,targettime,stoploss,target
0,1.11085,2024-10-01 08:40:00,2024-10-01 08:50:00,1.11105,1.11005
1,1.10890,2024-10-01 11:20:00,2024-10-01 11:40:00,1.10910,1.10810
2,1.10666,2024-10-01 13:35:00,2024-10-01 13:45:00,1.10686,1.10586
3,1.10334,2024-10-03 17:50:00,2024-10-03 19:05:00,1.10354,1.10254
4,1.09630,2024-10-07 09:10:00,2024-10-07 09:50:00,1.09650,1.09550
5,1.09702,2024-10-09 02:55:00,2024-10-09 04:50:00,1.09722,1.09622
6,1.09626,2024-10-09 06:55:00,2024-10-09 07:10:00,1.09646,1.09546
7,1.09620,2024-10-09 11:25:00,2024-10-09 11:55:00,1.09640,1.09540
8,1.09362,2024-10-10 11:10:00,2024-10-10 11:35:00,1.09382,1.09282
9,1.09364,2024-10-11 19:35:00,2024-10-13 22:00:00,1.09384,1.09284


In [31]:
c

,entryprice,DateTime,targettime,stoploss,target
0,1.11085,2024-10-01 08:40:00,2024-10-01 08:50:00,1.11105,1.11005
1,1.10890,2024-10-01 11:20:00,2024-10-01 11:40:00,1.10910,1.10810
2,1.10666,2024-10-01 13:35:00,2024-10-01 13:45:00,1.10686,1.10586
3,1.10334,2024-10-03 17:50:00,2024-10-03 19:05:00,1.10354,1.10254
4,1.09630,2024-10-07 09:10:00,2024-10-07 09:50:00,1.09650,1.09550
5,1.09702,2024-10-09 02:55:00,2024-10-09 04:50:00,1.09722,1.09622
6,1.09626,2024-10-09 06:55:00,2024-10-09 07:10:00,1.09646,1.09546
7,1.09620,2024-10-09 11:25:00,2024-10-09 11:55:00,1.09640,1.09540
8,1.09362,2024-10-10 11:10:00,2024-10-10 11:35:00,1.09382,1.09282
9,1.09364,2024-10-11 19:35:00,2024-10-13 22:00:00,1.09384,1.09284


In [20]:
result_df4=b
result_df4['loss']=result_df4['stoploss']-result_df4['entryprice']

result_df4['loss']=(result_df4['loss']*10000)

#result_df4

In [21]:
result_df4

,entryprice,DateTime,stoplosstime,stoploss,target,loss
0,1.11323,2024-09-30 19:55:00,2024-09-30 20:20:00,1.11343,1.11243,2.0
1,1.11319,2024-09-30 21:15:00,2024-09-30 21:50:00,1.11339,1.11239,2.0
2,1.10861,2024-10-01 10:30:00,2024-10-01 10:35:00,1.10881,1.10781,2.0
3,1.10917,2024-10-01 11:10:00,2024-10-01 11:15:00,1.10937,1.10837,2.0
4,1.10736,2024-10-01 14:50:00,2024-10-01 14:55:00,1.10756,1.10656,2.0
...,...,...,...,...,...,...
64,1.07920,2024-10-24 11:45:00,2024-10-24 11:50:00,1.07940,1.07840,2.0
65,1.08222,2024-10-25 01:20:00,2024-10-25 01:45:00,1.08242,1.08142,2.0
66,1.08317,2024-10-25 14:05:00,2024-10-25 14:10:00,1.08337,1.08237,2.0
67,1.08018,2024-10-25 17:20:00,2024-10-25 17:25:00,1.08038,1.07938,2.0


In [22]:
combined_df = c

combined_df['profit']=combined_df['entryprice']-combined_df['target']

combined_df['profit']=(combined_df['profit']*10000)
combined_df

,entryprice,DateTime,targettime,stoploss,target,profit
0,1.11085,2024-10-01 08:40:00,2024-10-01 08:50:00,1.11105,1.11005,8.0
1,1.10890,2024-10-01 11:20:00,2024-10-01 11:40:00,1.10910,1.10810,8.0
2,1.10666,2024-10-01 13:35:00,2024-10-01 13:45:00,1.10686,1.10586,8.0
3,1.10334,2024-10-03 17:50:00,2024-10-03 19:05:00,1.10354,1.10254,8.0
4,1.09630,2024-10-07 09:10:00,2024-10-07 09:50:00,1.09650,1.09550,8.0
5,1.09702,2024-10-09 02:55:00,2024-10-09 04:50:00,1.09722,1.09622,8.0
6,1.09626,2024-10-09 06:55:00,2024-10-09 07:10:00,1.09646,1.09546,8.0
7,1.09620,2024-10-09 11:25:00,2024-10-09 11:55:00,1.09640,1.09540,8.0
8,1.09362,2024-10-10 11:10:00,2024-10-10 11:35:00,1.09382,1.09282,8.0
9,1.09364,2024-10-11 19:35:00,2024-10-13 22:00:00,1.09384,1.09284,8.0


In [35]:
a

,entryprice,DateTime,entrytime,stoploss,target
0,1.11323,2024-09-30 19:55:00,2024-09-30 19:55:00,1.11343,1.11243
1,1.11319,2024-09-30 21:15:00,2024-09-30 21:15:00,1.11339,1.11239
2,1.11085,2024-10-01 08:40:00,2024-10-01 08:40:00,1.11105,1.11005
3,1.10861,2024-10-01 10:30:00,2024-10-01 10:30:00,1.10881,1.10781
4,1.10917,2024-10-01 11:10:00,2024-10-01 11:10:00,1.10937,1.10837
...,...,...,...,...,...
88,1.08222,2024-10-25 01:20:00,2024-10-25 01:20:00,1.08242,1.08142
89,1.08317,2024-10-25 14:05:00,2024-10-25 14:05:00,1.08337,1.08237
90,1.08018,2024-10-25 17:20:00,2024-10-25 17:20:00,1.08038,1.07938
91,1.07964,2024-10-25 19:20:00,2024-10-25 19:20:00,1.07984,1.07884


In [23]:
print(combined_df['profit'].sum())
print(result_df4['loss'].sum())

199.99999999997797
137.99999999998477


In [24]:
f=combined_df['profit'].sum()-result_df4['loss'].sum()
print('profit after loss',f)

profit after loss 61.99999999999321


In [ ]:
import time
from datetime import datetime, timedelta

while True:

    
    # Get the current time
    now = datetime.now()
    
    # Calculate the minutes to the next 5-minute multiple
    minutes_to_next_multiple_of_5 = (1 - (now.minute % 1)) % 1
    
    # Calculate the next run time at the next 5-minute multiple + 5 seconds
    next_run = (now + timedelta(minutes=minutes_to_next_multiple_of_5)).replace(second=1, microsecond=0)

    # If `next_run` is earlier than or equal to `now`, add 5 minutes to get the next 5-minute mark + 5 seconds
    if next_run <= now:
        next_run += timedelta(minutes=1)

    # Calculate the wait time in seconds
    wait_time = (next_run - now).total_seconds()

    # Wait until the next 5-minute multiple + 5 seconds
    time.sleep(wait_time)
    
    # Code to execute at the next 5-minute multiple
    print("Running at:", datetime.now())

    symbol="EURUSDm"
    timeframe=mt.TIMEFRAME_M1
    date_from=datetime(2024,10,27)
    date_to=datetime.now()
    #date_to=datetime(2024,10,30)
    
    
    df=pd.DataFrame(mt.copy_rates_range(symbol,timeframe,date_from,date_to))
    df['DateTime'] = pd.to_datetime(df['time'], unit='s')
    df=df.drop(columns=['tick_volume',	'spread','real_volume'])
   
   
    df = df.dropna()
    print(df['close'].tail(1).values[0])
    
    # Run the code
    preprocessor = get_data_transformer_object()

    preprocessor.fit(df)
    a, b, c = preprocessor.transform(df)
    # Extract the last datetime values from each DataFrame
    last_datetime_a = [i for i in c['DateTime'].values]
    last_datetime_b = [i for i in b['DateTime'].values]
    last_datetime_d =a['DateTime'].tail(1).values[0]
    tq=a['target'].tail(1).values[0]
    tq1=a['stoploss'].tail(1).values[0]
    tq2=a['entryprice'].tail(1).values[0]
    
    
    
    # Check if they are equal
    nop=len(mt.positions_get())
    print(nop)
    
    if nop==0:#last_datetime_d not in  last_datetime_b and last_datetime_d not in  last_datetime_a and nop==0 :
        print("The last datetime values are equal.")
        symbol="EURUSDm"
       # p=mt.symbol_info_tick("BTCUSDm").ask
        #p1=mt.symbol_info_tick("BTCUSDm").bid
        sl=tq1+0.0001
        t=tq+0.0001
        print("target:",tq)
        print("stoploss:",tq1)
        print("2entryt:",tq2)
        request={
            "action": mt.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.02,
            "type": mt.ORDER_TYPE_SELL,
            
            "price": mt.symbol_info_tick("EURUSDm").bid,
            "sl": sl,
            "tp": t,
       
            "comment": "python script close",
            "type_time": mt.ORDER_TIME_GTC,
            "type_filling": mt.ORDER_FILLING_IOC,
        }
        # send a trading request
        result=mt.order_send(request)
        result
        
    else:
        print("The last datetime values are not equal.")
    print("Code ran at:", datetime.now().strftime("%H:%M:%S"))


Running at: 2024-10-28 14:46:01.001642
1.08168
0
The last datetime values are equal.
target: 1.08085
stoploss: 1.08185
2entryt: 1.08165
Code ran at: 14:46:01
Running at: 2024-10-28 14:47:01.000476
1.08164
0
The last datetime values are equal.
target: 1.08085
stoploss: 1.08185
2entryt: 1.08165
Code ran at: 14:47:01
Running at: 2024-10-28 14:48:01.004050
1.08171
1
The last datetime values are not equal.
Code ran at: 14:48:01
Running at: 2024-10-28 14:49:01.000638
1.0815
1
The last datetime values are not equal.
Code ran at: 14:49:01


In [371]:
print(combined_df['profit'].sum()-result_df4['loss'].sum())


82260.0


In [189]:
result_df4

,DateTime,entry,target,stoploss,2entryt,loss,profit
0,2024-09-30 20:17:00,63651.88,63481.88,63721.88,2024-09-30 20:23:00,700030.0,1700000.0
2,2024-10-01 03:51:00,63690.99,63520.99,63760.99,2024-10-01 03:52:00,700030.0,1700000.0
7,2024-10-01 18:03:00,62150.11,61980.11,62220.11,2024-10-01 18:04:00,700030.0,1700000.0
8,2024-10-02 01:21:00,61246.59,61076.59,61316.59,2024-10-02 01:23:00,700030.0,1700000.0
9,2024-10-02 02:59:00,61789.03,61619.03,61859.03,2024-10-02 03:00:00,700030.0,1700000.0
...,...,...,...,...,...,...,...
257,2024-10-26 15:03:00,66759.12,66589.12,66829.12,2024-10-26 15:04:00,700030.0,1700000.0
259,2024-10-26 17:09:00,66953.25,66783.25,67023.25,2024-10-26 17:12:00,700030.0,1700000.0
260,2024-10-26 17:42:00,67047.08,66877.08,67117.08,2024-10-26 17:43:00,700030.0,1700000.0
263,2024-10-27 01:40:00,66957.82,66787.82,67027.82,2024-10-27 01:41:00,700030.0,1700000.0


In [38]:
combined_df['result']=1
result_df4['result']=0

In [41]:
merged_df['consecutive_zeros'] = (merged_df['result'] == 0).astype(int).groupby(merged_df['result'].ne(0).cumsum()).cumsum()

# Find the maximum number of consecutive zeros
max_consecutive_zeros = merged_df['consecutive_zeros'].max()
print("\nMaximum number of consecutive 0s:", max_consecutive_zeros)



Maximum number of consecutive 0s: 12


In [42]:
merged_df['consecutive_ones'] = (merged_df['result'] == 1).astype(int).groupby(merged_df['result'].ne(1).cumsum()).cumsum()

# Find the maximum number of consecutive zeros
max_consecutive_ones = merged_df['consecutive_ones'].max()
print("\nMaximum number of consecutive 1s:", max_consecutive_ones)


Maximum number of consecutive 1s: 4


In [40]:
merged_df = pd.concat([combined_df, result_df4])

# Sort the merged DataFrame by DateTime in ascending order
merged_df = merged_df.sort_values(by='DateTime').reset_index(drop=True)

#print(merged_df)

In [121]:
print(combined_df['profit'].sum())
print(result_df4['loss'].sum())

8592.000000000487
4199.999999999692


In [519]:
print(combined_df['profit'].sum())
print(result_df4['loss'].sum())

396.29999999996284
456.9499999999538


In [59]:
symbol="btcusf"
request={
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 0.02,
    "type": mt.ORDER_TYPE_SELL,
    
    "price": mt.symbol_info_tick("bt").bid,
    "sl": deviation,
    "tp":tp,
    "magic": 234000,
    "comment": "python script close",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,
}
# send a trading request
result=mt.order_send(request)

AttributeError: 'NoneType' object has no attribute 'bid'

In [60]:
symbol="btcusf"
position_id=mt.positions_get()[0]._asdict()['ticket']
request={
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 0.02,
    "type": mt.ORDER_TYPE_BUY,
    "position": position_id,
    "price": mt.symbol_info_tick("bt").ask,
    "sl": deviation,
    "tp":tp,
    "magic": 234000,
    "comment": "python script close",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,
}
# send a trading request
result=mt.order_send(request)

IndexError: tuple index out of range